In [ ]:
import pandas as pd
import numpy as np

import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text
from urllib.parse import quote_plus

import os
from dotenv import load_dotenv

#### Establish a connection between Python and the Sakila database.

In [ ]:
# load db credentials 

load_dotenv('db_credentials.env')

username = os.getenv('username')
password = quote_plus(os.getenv("password"))
host = os.getenv('host')
database = os.getenv('database')

In [ ]:
# create an sqlalchemy engine object -- sakila db

connection_string = f"mysql+pymysql://{username}:{password}@{host}/{database}"
engine_sakila = create_engine(connection_string)

#### Python Function 'rental_month'

In [ ]:
def rental_month (year,month,engine):

    '''
    this function takes desired year and month and database engine as parameters
    retrieve the rental data for the desired year and month
    return the rental data (as dataframe) for desired year and month
    '''
    
    rental_query = '''
            select *
            from sakila.rental
            '''
    with engine.connect() as connection:
        query = text(rental_query)
        result = connection.execute(query)

    rows = result.fetchall()
    columns = result.keys()
    rental_df = pd.DataFrame(rows,columns= columns)


    rental_df['rental_year'] = rental_df['rental_date'].dt.year
    rental_df['rental_month'] = rental_df['rental_date'].dt.month

    condition_year = (rental_df['rental_year'] == year)
    condition_month = (rental_df['rental_month'] == month)
    rental_df_filtered = rental_df[condition_year & condition_month]
    
    return rental_df_filtered



#### Python function 'rental_count_month'

In [ ]:


def rental_count_month(df,year,month):
    '''
    this function takes 3 parameter: rental data for desired year and month provided by rentals_month function, desired year, desired month
    display the number of rentals made by each customer_id during desired year and month
    return above displayed as a dataframe
    '''
    df_rental_month_count = df.groupby('customer_id')['rental_id'].agg('count').reset_index(name = f"rentals_{month}_{year}")
    return df_rental_month_count




#### Python function 'compare_rentals'

In [ ]:
def compare_rentals(df1,df2):
    '''
    this function takes 2 dataframes as paramters , 2 dataframes are provided by rental_count_month function
    it combines 2 dataframes and create a column 'difference', which is the diff of rental counts in two months
    return combined dataframe
    '''
    
    df_combined = pd.merge(df1,df2,how = 'inner',on = 'customer_id')

    column1 = df1.columns[1]
    column2 = df2.columns[1]
    df_combined['difference'] = df_combined[column1] - df_combined[column2]

    return df_combined


#### call above functions
#### output is a dataframe showing the rental counts difference in different months for each customer

In [ ]:
df_5 = rental_month(2005,5,engine_sakila)
df5 = rental_count_month(df_5,2005,5)

df_6 = rental_month(2005,6,engine_sakila)
df6 = rental_count_month(df_6,2005,6)

df_rental_combined = compare_rentals(df5,df6)
df_rental_combined